In [4]:
import json

import requests
from dotenv import load_dotenv
from pydantic import BaseModel, Field

from wow_api.external.blizz_api.auth import BlizzardApiAuth

load_dotenv()

True

In [2]:
api_auth = BlizzardApiAuth()
token = api_auth.get_token()

made new token: access_token='USOG7QkKNVolljRksvNfnyD9HL6SoQrQ97' token_type='bearer' expires_in=datetime.timedelta(seconds=86399) sub='e99ce1a2c0454313b5f5b95d5cce9901' expires_at=datetime.datetime(2023, 4, 5, 17, 51, 40, 582587, tzinfo=datetime.timezone.utc)
access_token='USOG7QkKNVolljRksvNfnyD9HL6SoQrQ97' token_type='bearer' expires_in=datetime.timedelta(seconds=86399) sub='e99ce1a2c0454313b5f5b95d5cce9901' expires_at=datetime.datetime(2023, 4, 5, 17, 51, 40, 582587, tzinfo=datetime.timezone.utc)


In [3]:
class ConnectedRealm(BaseModel):
    href: str


class ConnectedRealmsIndexResponse(BaseModel):
    connected_realms: list[ConnectedRealm]


In [6]:
url = 'https://us.api.blizzard.com/data/wow/connected-realm/index?namespace=dynamic-us&locale=en_US'

connected_realms_res = requests.get(url, headers= {
        'Authorization': f'Bearer {token.access_token}'
    })

connected_realms = ConnectedRealmsIndexResponse.parse_obj(connected_realms_res.json())

with open('./out/connected-realms.json', 'w') as realms_file:
    realms_file.write(connected_realms.json(indent=2))

with open('./out/raw-connected-realms.json', 'w') as file:
    json.dump(connected_realms_res.json(), file, indent=2)

In [7]:
realm_res = requests.get(connected_realms.connected_realms[0].href, headers= {
        'Authorization': f'Bearer {token.access_token}'
    })

connected_realm = ConnectedRealmResponse.parse_obj(realm_res.json())

with open(f'./out/realms/{connected_realm.id}.json', 'w') as file:
    file.write(connected_realm.json(indent=2))

In [10]:
class Key(BaseModel):
    href: str


class CurrentLeaderboard(BaseModel):
    key: Key
    name: LocaleText
    id: int


class MythicLeaderboardsResponse(BaseModel):
    current_leaderboards: list[CurrentLeaderboard]

In [11]:
mythic_leaderboards_res = requests.get(connected_realm.mythic_leaderboards.href, headers= {
        'Authorization': f'Bearer {token.access_token}'
    })

leaderboards = MythicLeaderboardsResponse.parse_obj(mythic_leaderboards_res.json())

In [12]:
leaderboards.current_leaderboards[0].key.href

'https://us.api.blizzard.com/data/wow/connected-realm/121/mythic-leaderboard/2/period/900?namespace=dynamic-us'

In [34]:






class Map(BaseModel):
    name: LocaleText
    id: int





class Realm(BaseModel):
    key: Key
    id: int
    slug: str


class Profile(BaseModel):
    name: str
    id: int
    realm: Realm


class Faction(BaseModel):
    type: str


class Specialization(BaseModel):
    key: Key
    id: int


class Member(BaseModel):
    profile: Profile
    faction: Faction
    specialization: Specialization


class Color(BaseModel):
    r: int
    g: int
    b: int
    a: float


class MythicRating(BaseModel):
    color: Color
    rating: float


class LeadingGroup(BaseModel):
    ranking: int
    duration: int
    completed_timestamp: int
    keystone_level: int
    members: list[Member]
    mythic_rating: MythicRating


class KeystoneAffix(BaseModel):
    key: Key
    name: LocaleText
    id: int


class CurrentAffix(BaseModel):
    keystone_affix: KeystoneAffix
    starting_level: int


class KeyLeaderboardResponse(BaseModel):
    map: Map
    period: int
    period_start_timestamp: int
    period_end_timestamp: int
    connected_realm: ConnectedRealm
    leading_groups: list[LeadingGroup]
    keystone_affixes: list[CurrentAffix]
    map_challenge_mode_id: int
    name: LocaleText

In [35]:
key_leaderboard_res = requests.get('https://us.api.blizzard.com/data/wow/connected-realm/121/mythic-leaderboard/2/period/899?namespace=dynamic-us', headers= {
        'Authorization': f'Bearer {token.access_token}'
    })

In [36]:
key_leaderboard = KeyLeaderboardResponse.parse_obj(key_leaderboard_res.json())

In [38]:
key_leaderboard.leading_groups[0].members[0].profile.name


'Gokgokgokgok'

In [51]:
res = requests.get('https://us.api.blizzard.com/data/wow/playable-race/29?namespace=static-10.0.7_48520-us',
                        headers= {
                            'Authorization': f'Bearer {token.access_token}'
                        })

In [52]:
res.status_code

200

In [53]:
with open('./data/race-res.json', 'w') as spec_file:
    spec_file.write(
        res.text
    )
